In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from google.colab import drive
import os
import pickle
from scipy.signal import find_peaks, argrelextrema
from scipy.spatial.distance import cdist

In [2]:
def RK4step(func, current_state, t, dt):
    k1 = func(t, current_state)
    k2 = func(t + 0.5 * dt, current_state + 0.5 * dt * k1)
    k3 = func(t + 0.5 * dt, current_state + 0.5 * dt * k2)
    k4 = func(t + dt, current_state + dt * k3)
    return current_state + (dt / 6) * (k1 + 2 * k2 + 2 * k3 + k4)

# Lorenz system - Drive

\begin{align*}
\frac{dx}{dt} &= \sigma (y - x) \\
\frac{dy}{dt} &= x (\rho - z) - y \\
\frac{dz}{dt} &= xy - \beta z
\end{align*}

In [3]:
class LorenzSystem:

    def __init__(self):
        self.sigma = 10.0
        self.rho = 28.0
        self.beta = 8.0 / 3.0
        self.state = np.array([1, 1, 1])

    def __call__(self, t, u):
        return np.array([
            self.sigma * (u[1] - u[0]),
            u[0] * (self.rho - u[2]) - u[1],
            u[0] * u[1] - self.beta * u[2]
        ])

    def reinitialise_ic(self):
        self.state = np.array([1, 1, 1])

    def randomise_ic(self):
        self.state = 2 * (np.random.rand(self.state.size) - 0.5)

# Reservoir System - Response
\begin{align*}
\frac{d}{dt}\mathbf{r}(t) = \gamma \left[ -r(t) + \tanh \left( M r(t) + \sigma W_{\text{in}} u(t) \right) \right]
\end{align*}

In [4]:
class ReservoirComputer:
    def __init__(self, drive_system, reservoir_size, connectivity, spectral_radius, gamma, sigma):
        self.rng_M = np.random.default_rng(1911)
        self.rng_Win = np.random.default_rng(1912)
        self.rng_state = np.random.default_rng(1913)

        self.r = 2 * (self.rng_state.random(reservoir_size) - 0.5)
        self.u = drive_system
        self.spectral_radius = spectral_radius
        self.connectivity = connectivity
        self.reservoir_size = reservoir_size

        self.M, self.original_M, self.sparsification_mask = self.generate_matrix_and_mask(reservoir_size, connectivity, spectral_radius)
        self.Win = self.generate_Win(reservoir_size, self.u.state.size)

        self.sigma = sigma
        self.gamma = gamma
        self.res_update_func_listen = self.reservoir_update_equation
        self.res_update_func_pred = self.reservoir_update_equation_prediction

        self.dt = 0.01
        self.t = 0.0
        self.beta = 0.001
        self.q = lambda x: np.concatenate((np.array(x), np.array(x)**2))
        self.A = None
        self.b = None
        self.phi = None

    def generate_matrix_and_mask(self, reservoir_size, connectivity, spectral_radius):
        M = 2 * (self.rng_M.random((reservoir_size, reservoir_size)) - 0.5)
        sparsification_mask = self.rng_M.binomial(1, connectivity, size=(reservoir_size, reservoir_size))
        original_M = M.copy()
        sparsification_mask_copy = sparsification_mask.copy()
        M *= sparsification_mask
        M *= spectral_radius / max(np.abs(np.linalg.eigvals(M))).real
        return M, original_M, sparsification_mask_copy

    def generate_Win(self, reservoir_size, input_size):
        Win = np.zeros((reservoir_size, input_size))
        random_cols = self.rng_Win.integers(0, input_size, reservoir_size)
        random_values = self.rng_Win.uniform(-1, 1, reservoir_size)
        Win[np.arange(reservoir_size), random_cols] = random_values
        return Win

    def reservoir_update_equation(self, t, r):
        return self.gamma * (-r + np.tanh(self.M @ r + self.sigma * self.Win @ self.u.state))

    def reservoir_update_equation_prediction(self, t, r):
        return self.gamma * (-r + np.tanh(self.M @ r + self.sigma * self.Win @ self.phi(self.q(r))))

    def listen(self, t_listen):
        num_timesteps = int(t_listen / self.dt)

        for _ in range(num_timesteps):
            self.u.state = RK4step(self.u, self.u.state, self.t, self.dt)
            self.r = RK4step(self.res_update_func_listen, self.r, self.t, self.dt)
            self.t += self.dt

    def train(self, t_train):
        num_timesteps = int(t_train / self.dt)

        u_states = []
        symmetry_broken_r_states = []
        for _ in range(num_timesteps):
            self.u.state = RK4step(self.u, self.u.state, self.t, self.dt)
            u_states.append(self.u.state)
            self.r = RK4step(self.res_update_func_listen, self.r, self.t, self.dt)
            symmetry_broken_r_states.append(self.q(self.r))
            self.t += self.dt

        u_states = np.array(u_states)
        symmetry_broken_r_states = np.array(symmetry_broken_r_states)
        self.parameterise_phi(symmetry_broken_r_states, u_states)

    def predict(self, t_predict):
        num_timesteps = int(t_predict / self.dt)

        predictions = [self.phi(self.q(self.r))]
        for _ in range(num_timesteps):
            self.r = RK4step(self.res_update_func_pred, self.r, self.t, self.dt)
            prediction = self.phi(self.q(self.r))
            predictions.append(prediction)
            self.t += self.dt

        return np.array(predictions)

    def parameterise_phi(self, X, Y):
        n = X.shape[0]
        dims_x = X.shape[1]
        dims_y = Y.shape[1]
        A = np.linalg.inv(X.T @ X + self.beta * np.identity(dims_x)) @ X.T @ Y
        b = (1 / n) * np.sum(Y - X @ A, axis=0)
        self.A = A
        self.b = b
        self.phi = lambda x: self.A.T @ x + self.b
        #self.phi = lambda x: x @ self.A + self.b

    def set_spectral_radius(self, spectral_radius):
        if spectral_radius == 0:
            self.M = np.zeros_like(self.original_M)
        else:
            self.M = (self.original_M * self.sparsification_mask) * (spectral_radius / max(np.abs(np.linalg.eigvals(self.original_M * self.sparsification_mask))).real)
        self.spectral_radius = spectral_radius

    def time_ts(self, natural_time):
        return int(natural_time/self.dt)

    def reinitialise_same_RC(self):
        self.reinitialise_state()
        self.u.reinitialise_ic()
        self.A = None
        self.b = None
        self.phi = None

    def reinitialise_state(self):
        self.r = 2 * (self.rng_state.random(self.reservoir_size) - 0.5)
        self.t = 0.0

    def reinitialise_M(self):
        self.M, self.original_M, self.sparsification_mask = self.generate_matrix_and_mask(self.reservoir_size, self.connectivity, self.spectral_radius)

    def reinitialise_Win(self):
        self.Win = self.generate_Win(self.reservoir_size, self.u.state.size)

    def reinitialise_RC(self):
        self.reinitialise_M()
        self.reinitialise_Win()
        self.reinitialise_state()

## The following function `time_ts`converts the time parameter into timestep units

In [5]:
def time_ts(natural_time):
        return int(natural_time/0.01)

# Object for RC simulation in bulk.

In [6]:
class ReservoirComputer_BulkSimulation(ReservoirComputer):
    def __init__(self, drive_system, reservoir_size, connectivity, spectral_radius, gamma, sigma):
        super().__init__(drive_system, reservoir_size, connectivity, spectral_radius, gamma, sigma)

    def predict_100(self, t_predict):
        predictions_100 = []
        continuation_states = []
        for _ in range(100):
            self.reinitialise_state()
            predictions = self.predict(t_predict)
            predictions_100.append(predictions)
            continuation_states.append(self.r)

        print("Predictions complete")
        return np.array(predictions_100), np.array(continuation_states)

    def simulate_100(self, t_listen, t_train, t_predict):
        self.rng_state = np.random.default_rng(1913)
        self.reinitialise_same_RC()

        self.listen(t_listen)
        self.train(t_train)
        trajectories, continuation_states = self.predict_100(t_predict)
        return np.array(trajectories), np.array(continuation_states)

    # Here is the adjustment for continuations
    def predict_from_continuation_100(self, t_predict, previous_continuation_states):
        predictions_100 = []
        next_continuation_states = []
        for i in range(100):
            self.r = previous_continuation_states[i,]
            predictions = self.predict(t_predict)
            predictions_100.append(predictions)
            next_continuation_states.append(self.r)

        print("Predictions complete")
        return np.array(predictions_100), np.array(next_continuation_states)

    def continuation_100(self, t_listen, t_train, t_predict, previous_continuation_states):
        self.rng_state = np.random.default_rng(1913)
        self.reinitialise_same_RC()

        self.listen(t_listen)
        self.train(t_train)
        trajectories, next_continuation_states = self.predict_from_continuation_100(t_predict, previous_continuation_states)
        return np.array(trajectories), np.array(next_continuation_states)

# Perform 10 simulations of 100 trajectories for each spectral radius. For each of the 10 randomise M 5 times, and Win 5 times.

Structure of stored simulations, `tuple = (np.array(trajectories), np.array(seeds))`



In [7]:
lorenz = LorenzSystem()                 # U(t)
reservoir_size = 100                    # N
connectivity = 0.05                     # P
spectral_radius = 0.6                   # ρ
gamma = 10.0                            # γ
sigma = 0.2                             # σ

In [8]:
params = (lorenz, reservoir_size, connectivity, spectral_radius, gamma, sigma)

In [9]:
times = (100, 100, 100)

In [12]:
def generate_spectral_radii_list(start, end, steps):
  spectral_radii = np.linspace(start, end, steps)
  spectral_radii = [np.round(p, 6) for p in spectral_radii]
  spectral_radii = [str(p) for p in spectral_radii]
  return spectral_radii

In [ ]:
# spectral_radii_Lorenz_birth = generate_spectral_radii_list(0.0, 0.005, 55)
# spectral_radii = [str(p) for p in spectral_radii_Lorenz_birth]

#Simulation from 9th instance of 0.004 down. to 0.002

- Load the seed from the 9th (index = 8) seed for 0.004 and run simulations from that down to o.002 use small jumps to capture 40 steps.

- run again from 0.00365 to 0.00355

In [14]:
spectral_radii_Lorenz_birth = generate_spectral_radii_list(0.00365, 0.00350, 60)
spectral_radii = [str(p) for p in spectral_radii_Lorenz_birth]
spectral_radii

['0.00365',
 '0.003647',
 '0.003645',
 '0.003642',
 '0.00364',
 '0.003637',
 '0.003635',
 '0.003632',
 '0.00363',
 '0.003627',
 '0.003625',
 '0.003622',
 '0.003619',
 '0.003617',
 '0.003614',
 '0.003612',
 '0.003609',
 '0.003607',
 '0.003604',
 '0.003602',
 '0.003599',
 '0.003597',
 '0.003594',
 '0.003592',
 '0.003589',
 '0.003586',
 '0.003584',
 '0.003581',
 '0.003579',
 '0.003576',
 '0.003574',
 '0.003571',
 '0.003569',
 '0.003566',
 '0.003564',
 '0.003561',
 '0.003558',
 '0.003556',
 '0.003553',
 '0.003551',
 '0.003548',
 '0.003546',
 '0.003543',
 '0.003541',
 '0.003538',
 '0.003536',
 '0.003533',
 '0.003531',
 '0.003528',
 '0.003525',
 '0.003523',
 '0.00352',
 '0.003518',
 '0.003515',
 '0.003513',
 '0.00351',
 '0.003508',
 '0.003505',
 '0.003503',
 '0.0035']

In [15]:
from google.colab import drive
drive.mount('/content/drive')

def load_dictionary(file_path):
    with open(file_path, 'rb') as file:
        loaded_dictionary = pickle.load(file)
    print(f"Data loaded successfully from {file_path}")
    return loaded_dictionary

Mounted at /content/drive


In [16]:
file_path = f'drive/My Drive/Lorenz_birth_data/0.004_M.pkl'

In [17]:
_0_0_0_4_data = load_dictionary(file_path)

Data loaded successfully from drive/My Drive/Lorenz_birth_data/0.004_M.pkl


In [18]:
_, seeds = _0_0_0_4_data
seed = seeds[8]

In [19]:
def RC_regen_from_seed(params, seed, randomised_parameter):
  RC_B = ReservoirComputer_BulkSimulation(*params)
  if randomised_parameter == "M":
    RC_B.rng_M.bit_generator.state = seed
    RC_B.reinitialise_M()
  else:
    RC_B.rng_Win.bit_generator.state = seed
    RC_B.reinitialise_Win()
  return RC_B

In [20]:
RC_B = RC_regen_from_seed(params, seed, "M")

trajectories_list = []
continuation_states = []

for i, p in enumerate(spectral_radii):

    RC_B.set_spectral_radius(float(p))
    # just do normal simulation for first parameter as no continuation states to be passed in
    if i == 0:
        trajectories, continuation_states = RC_B.simulate_100(*times)
    else:
        trajectories, continuation_states = RC_B.continuation_100(*times, continuation_states)

    print(trajectories.shape)
    # retrieve last state from each trajectory to be stored as the states for continuation in next rho
    print(continuation_states.shape)

    # append the attractor to a list to be stored
    trajectories_list.append(trajectories)

    print(f"\033[1mSimulation complete for ρ = {p} \033[0m")

result = np.array(trajectories_list)
bifurcation_analysis_data_piece = result

Predictions complete
(100, 10001, 3)
(100, 100)
Simulation complete for ρ = 0.00365 
Predictions complete
(100, 10001, 3)
(100, 100)
Simulation complete for ρ = 0.003647 
Predictions complete
(100, 10001, 3)
(100, 100)
Simulation complete for ρ = 0.003645 
Predictions complete
(100, 10001, 3)
(100, 100)
Simulation complete for ρ = 0.003642 
Predictions complete
(100, 10001, 3)
(100, 100)
Simulation complete for ρ = 0.00364 
Predictions complete
(100, 10001, 3)
(100, 100)
Simulation complete for ρ = 0.003637 
Predictions complete
(100, 10001, 3)
(100, 100)
Simulation complete for ρ = 0.003635 
Predictions complete
(100, 10001, 3)
(100, 100)
Simulation complete for ρ = 0.003632 
Predictions complete
(100, 10001, 3)
(100, 100)
Simulation complete for ρ = 0.00363 
Predictions complete
(100, 10001, 3)
(100, 100)
Simulation complete for ρ = 0.003627 
Predictions complete
(100, 10001, 3)
(100, 100)
Simulation complete for ρ = 0.003625 
Predictions complete
(100, 10001, 3)
(100, 100)
Simulatio

In [21]:
# from google.colab import drive
# drive.mount('/content/drive')

def save_dictionary(dictionary, file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    with open(file_path, 'wb') as file:
        pickle.dump(dictionary, file, protocol=pickle.HIGHEST_PROTOCOL)
    print(f'Dictionary saved to {file_path}')

# def save_dictionary_to_drive(dictionary, file_path):
#     full_path = os.path.join('/content/drive/MyDrive', file_path)
#     directory = os.path.dirname(full_path)
#     if not os.path.exists(directory):
#         os.makedirs(directory)
#     with open(full_path, 'wb') as file:
#         pickle.dump(dictionary, file, protocol=pickle.HIGHEST_PROTOCOL)
#     print(f'Dictionary saved to {full_path}')

# def load_dictionary(file_path):
#     with open(file_path, 'rb') as file:
#         loaded_dictionary = pickle.load(file)
#     print(f"Data loaded successfully from {file_path}")
#     return loaded_dictionary

In [22]:
#file_path = f'drive/My Drive/Lorenz_birth_data/poor_mans_continuation_Lorenz_birth_0.004_to_0.002.pkl'
file_path = f'drive/My Drive/Lorenz_birth_data/poor_mans_continuation_Lorenz_birth_0.00365_to_0.00350.pkl'
# file_path = f'bifurcation_analysis_data/{p}_{randMat}.pkl'
file_path

'drive/My Drive/Lorenz_birth_data/poor_mans_continuation_Lorenz_birth_0.00365_to_0.00350.pkl'

In [23]:
save_dictionary(bifurcation_analysis_data_piece, file_path)

Dictionary saved to drive/My Drive/Lorenz_birth_data/poor_mans_continuation_Lorenz_birth_0.00365_to_0.00350.pkl
